In [1]:
# train.py
import os
import time
import torch
import torch.nn as nn
from torch.autograd import Variable
from preprocess import questiongenerator, imagefinder

train_path = 'text/train_balanced_questions.json'
image_path = 'images/'
val_path = 'validation/val_balanced_questions.json'
batch_size = 16
questionloader = questiongenerator(train_path, batch_size)
valiloader = questiongenerator(val_path, batch_size)

def validation(valiloader, net):
    acc_sum, n = 0.0, 0
    for img_no, q, a in valiloader:
        img = imagefinder(img_no)
        a_hat = net(img, q)
        acc_sum += (a_hat.argmax(dim = 1) == a).sum().item()
        n += a.shape[0]
    return acc_sum/n

def train(GQA, batch_size, num_epochs):
    optimizer = torch.optim.SGD(GQA.parameters(),lr = 0.1)
    loss = nn.CrossEntropyLoss()
    max_score = 0
    for i in range(num_epochs):
        loss_sum = 0.0
        acc_sum = 0.0
        n = 0
        start = time.time()
        for (img_no, q, a) in questionloader:
            img = imagefinder(image_path, img_no)
            a_hat = GQA(img, q)
            a = a.squeeze_()
            l = loss(a_hat, a)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            loss_sum += l.item()
            acc_sum += (a_hat.argmax(dim = 1) == a)
            n += a.shape[0]
            if n % 4 == 0:
                print('#',n,'time for 4 questions:',time.time()-start,'s')
        vali_acc = validation(valiloader,GQA)
        print('Epoch #%d, Loss:.3f, Train Acc: %.4f, Validation Acc:%.4f'%(i,loss_sum,acc_sum/n,vali_acc))
        

In [2]:
from model import GQA
net = GQA(1024)
train(net,2,1)

can not find sofa can not find sof
can not find spatula can not find spatul
can not find umbrellas can not find motorcycles 

RuntimeError: 1D target tensor expected, multi-target not supported

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input)
target = torch.empty(3, dtype=torch.long).random_(5)
print(target)
output = loss(input, target)
output.backward()